In [1]:
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell
from tensorflow.python.ops.rnn_cell import BasicRNNCell,BasicLSTMCell,GRUCell
import numpy as np

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/", one_hot=True)
print ("MNIST loaded")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}
print ("parameters ready")

In [3]:
#def RNN(x, weights, biases):
with tf.variable_scope("model"):
    #tf.get_variable_scope().reuse_variables()

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Permuting batch_size and n_steps
    x_t = tf.transpose(x, [1, 0, 2])
    # Reshaping to (n_steps*batch_size, n_input)
    x_t = tf.reshape(x_t, [-1, n_input])
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x_t = tf.split(0, n_steps, x_t)

    # Define a rnn cell with tensorflow
    cell = rnn_cell.BasicRNNCell(n_hidden) # 다른 셀로 교체 가능 : BasicRNNCell,BasicLSTMCell,GRUCell

    # Get lstm cell output
    outputs, states = rnn.rnn(cell, x_t, dtype=tf.float32) # RNN 실행

    # Linear activation, using rnn inner loop last output
    pred = tf.matmul(outputs[-1], weights['out']) + biases['out'] # output의 마지막 값을 사용

#pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


print ("Network ready")

In [4]:
# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))

        # Run optimization op (backprop)
        _, acc, loss = sess.run([optimizer,accuracy,cost], feed_dict={x: batch_x, y: batch_y})
        
        if step % display_step == 0:
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label})

Iter 1280, Minibatch Loss= 1.900115, Training Accuracy= 0.27344
Iter 2560, Minibatch Loss= 1.296756, Training Accuracy= 0.56250
Iter 3840, Minibatch Loss= 1.350010, Training Accuracy= 0.59375
Iter 5120, Minibatch Loss= 0.950676, Training Accuracy= 0.67969
Iter 6400, Minibatch Loss= 0.674714, Training Accuracy= 0.75000
Iter 7680, Minibatch Loss= 1.173815, Training Accuracy= 0.67188
Iter 8960, Minibatch Loss= 0.779203, Training Accuracy= 0.73438
Iter 10240, Minibatch Loss= 0.714008, Training Accuracy= 0.74219
Iter 11520, Minibatch Loss= 0.787130, Training Accuracy= 0.77344
Iter 12800, Minibatch Loss= 0.891985, Training Accuracy= 0.71875
Iter 14080, Minibatch Loss= 0.528170, Training Accuracy= 0.81250
Iter 15360, Minibatch Loss= 0.396232, Training Accuracy= 0.87500
Iter 16640, Minibatch Loss= 0.545354, Training Accuracy= 0.82812
Iter 17920, Minibatch Loss= 0.604140, Training Accuracy= 0.79688
Iter 19200, Minibatch Loss= 0.469294, Training Accuracy= 0.82812
Iter 20480, Minibatch Loss= 0.29